In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

# --- PARÁMETROS QUE PUEDES AJUSTAR ---
IMAGE_SIZE = 128
BATCH_SIZE = 64
DATASET_PATH = "C:/Users/ivcho/Desktop/Universidad/Proyecto_GAN/dataset/VincentVanGogh"

print("Empezando a cargar y preprocesar las imágenes de todas las subcarpetas")

processed_images = []

#Para cada carpeta, nos da su ruta (dirpath) y la lista de archivos (filenames).
for dirpath, dirnames, filenames in os.walk(DATASET_PATH):
    for filename in filenames:
        #Construir ruta completa al archivo
        path = os.path.join(dirpath, filename)

        #Leer imagen con OpenCV
        image = cv2.imread(path)

        # La convertimos de BGR (formato de OpenCV) a RGB (formato estándar)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        #Redimensionamos el tamaño de la imagen
        image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))

        processed_images.append(image)

#Convertir lista de imágenes procesadas en un array de NumPy
X_train = np.array(processed_images)

print("Valores no normalizados de la imagen [0, 255]")
una_imagen = X_train[1]
print(f"La forma de una sola imagen es: {una_imagen.shape}")
print("Estos son los valores de los 3 canales de color (R, G, B) para el píxel en la esquina superior izquierda (coordenada 0,0):")
pixel_esquina = una_imagen[0][0]
print(f"{pixel_esquina}\n")


#Los modelos GAN funcionan mejor con valores entre 1 y -1, por tanto se proceden a normalizar los valores.
print("Valores normalizados de la [-1, 1]")
X_train = (X_train.astype(np.float32) - 127.5) / 127.5

una_imagen = X_train[1]

print(f"La forma de una sola imagen es: {una_imagen.shape}")

print("Estos son los valores de los 3 canales de color (R, G, B) para el píxel en la esquina superior izquierda (coordenada 0,0):")
pixel_esquina = una_imagen[0][0]
print(f"{pixel_esquina}\n")

print(f"Hecho, se han cargado {len(X_train)} imágenes.")
print(f"La forma de nuestro set es {X_train.shape}")

# El problema es que matplotlib es como un humano, no entiende el "idioma" de la IA.
# Para mostrar una imagen correctamente, espera que los valores de los píxeles estén en uno de estos dos rangos:
plt.imshow((X_train[1] + 1) / 2) # Se deshace la normalización.
plt.axis("off")
plt.show()

#Convertir dataset en un formato eficiente para TensorFlow
train_dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(len(X_train)).batch(BATCH_SIZE)

        

In [ ]:
from tensorflow.keras import layers

# Definimos el tamaño del vector de ruido. Será la entrada del generador.
# Piensa en esto como la "complejidad" de la semilla de inspiración. 100 es un valor estándar.
NOISE_DIM = 100

def build_generator():
    model = keras.Sequential(name="Generador")
     # --- Capa de Entrada: Proyectar el ruido inicial ---
    # Tomamos el vector de ruido de 100 dimensiones y lo proyectamos a un espacio más grande.
    # 8*8*256 es el tamaño de la primera "tela" sobre la que empezará a pintar.
    model.add(layers.Dense(8 * 8 * 256, input_dim=NOISE_DIM))
    # Usamos LeakyReLU, una activación que funciona muy bien en GANs.
    model.add(layers.LeakyReLU(alpha=0.2))

    # --- Darle forma de imagen pequeña ---
    # Convertimos el vector en un cubo de (8, 8, 256)
    model.add(layers.Reshape((8, 8, 256)))

    # --- Proceso de Upsampling (Hacer la imagen más grande) ---
    # Vamos a ir doblando el tamaño de la imagen y reduciendo el número de filtros.
    # Capa 1: de 8x8 a 16x16
    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))

    # Capa 2: de 16x16 a 32x32
    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))

    # Capa 3: de 32x32 a 64x64
    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))

    # Capa 4: de 64x64 a 128x128
    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))

    # --- Capa de Salida: La pintura final ---
    # La última capa debe tener 3 canales (R, G, B)
    # Usamos la activación 'tanh' porque su salida está en el rango [-1, 1],
    # ¡exactamente el mismo rango en el que normalizamos nuestras imágenes!
    model.add(layers.Conv2D(3, (5, 5), padding='same', activation='tanh'))
    
    return model

# Construimos el generador
generator = build_generator()
print("--- ARQUITECTURA DEL GENERADOR ---")
generator.summary()

In [ ]:
def build_discriminator():
    model = keras.Sequential(name="Discriminador")

    #Capa de entrada 128x128x3
    model.add(layers.Conv2D(64, (5,5), strides=(2,2), padding="same", input_shape=[IMAGE_SIZE, IMAGE_SIZE, 3]))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    #Capas ocultas
    #64x64x3
    model.add(layers.Conv2D(128, (5,5), strides=(2,2), padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    #32x32x3
    model.add(layers.Conv2D(256, (5,5), strides=(2,2), padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    #16x16x3
    #Capa de Salida
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation="sigmoid"))
    return model

discriminator = build_discriminator()
print("--- ARQUITECTURA DEL GENERADOR ---")
discriminator.summary()

In [ ]:
import time

# La función de pérdida mide qué tan equivocados están los modelos.
# Usamos BinaryCrossentropy porque es un problema de clasificación binaria (0 o 1 / Falso o Real).
# from_logits=True es una recomendación técnica para que los cálculos sean más estables.
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)

def discriminator_loss(real_output, fake_output):
     # El crítico quiere que las imágenes REALES se clasifiquen como 1.
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    
    # El crítico quiere que las imágenes FALSAS se clasifiquen como 0
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    # El generador quiere ENGAÑAR al crítico, es decir,
    # quiere que sus imágenes FALSAS se clasifiquen como 1.
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer =  tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
@tf.function
def train_step(images):
    #1. Generamos ruido para dárselo de comer al generador
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])
    
    # 2. Usamos GradientTape para "grabar" las operaciones y poder calcular los gradientes.
    # Necesitamos dos "grabadoras", una para cada modelo.
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # 3. El generador crea imágenes falsas a partir del ruido.
        generated_images = generator(noise, training=True)

        # 4. El discriminador juzga tanto las imágenes reales como las falsas.
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        # 5. Calculamos las pérdidas para cada uno usando las funciones que definimos antes.
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

        # Para las imágenes reales, el objetivo es 1.
        # Redondeamos las predicciones y vemos cuántas son iguales a 1.
        real_acc = tf.reduce_mean(tf.cast(tf.equal(tf.round(real_output), 1.0), tf.float32))
        
        # Para las imágenes falsas, el objetivo es 0.
        # Redondeamos las predicciones y vemos cuántas son iguales a 0.
        fake_acc = tf.reduce_mean(tf.cast(tf.equal(tf.round(fake_output), 0.0), tf.float32))
        
        # La precisión total es la media de las dos.
        total_acc = (real_acc + fake_acc) / 2.0

    # 6. Calculamos los gradientes (cómo debe cambiar cada peso para mejorar).
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    # 7. Aplicamos esos cambios a los modelos usando los optimizadores.
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss, total_acc

In [ ]:
import csv

EPOCHS = 500 # Puedes empezar con 50-100 para probar, 200-500 para mejores resultados.
# Usaremos siempre el mismo lote de ruido para generar las imágenes de muestra.
# Así podremos ver la evolución del arte del generador desde la misma "idea" inicial.
seed = tf.random.normal([16, NOISE_DIM])

# Asegúrate de crear esta carpeta en el mismo lugar donde tienes tu notebook
os.makedirs('gan_images', exist_ok=True)

def generate_and_save_images(model, epoch, test_input):
    # `training` se pone a False para que el modelo se comporte en modo inferencia.
    predictions = model(test_input, training=False)
    
    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        # Desnormalizamos la imagen para poder visualizarla correctamente
        plt.imshow((predictions[i, :, :, :] + 1) / 2.0)
        plt.axis('off')

    # Guardamos la figura.
    plt.savefig('results/v1/gan_images_v1/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

def train(dataset, epochs):
    # Creamos el archivo para los logs de entrenamiento
    log_file = 'logs/training_log_v1.csv'
    with open(log_file, 'w', newline='') as f:
        writer = csv.writer(f)
        # Escribimos los títulos de nuestras columnas
        writer.writerow(['Epoch', 'Generator_Loss', 'Discriminator_Loss', 'Discriminator_Accuracy'])
        
    for epoch in range(epochs):
        start = time.time()
        
        # Variables para guardar la media de las métricas de la época
        total_gen_loss = 0
        total_disc_loss = 0
        total_acc = 0
        num_batches = 0
        
        # Por cada lote de imágenes en nuestro dataset...
        for image_batch in dataset:
            gen_loss, disc_loss, acc = train_step(image_batch)
            total_gen_loss += gen_loss
            total_disc_loss += disc_loss
            total_acc += acc
            num_batches += 1

        # Calculamos la media de la época
        avg_gen_loss = total_gen_loss / num_batches
        avg_disc_loss = total_disc_loss / num_batches
        avg_acc = total_acc / num_batches
        
        # Imprimimos los resultados de la época de forma clara
        print(f'Época {epoch + 1} | Pérdida Gen: {avg_gen_loss:.4f} | Pérdida Disc: {avg_disc_loss:.4f} | Precisión Disc: {avg_acc:.2%}')

        # Guardamos los resultados de esta época en el archivo CSV.
        with open(log_file, 'a', newline='') as f:
            writer = csv.writer(f)
            # Los valores de loss/acc son tensores de TensorFlow, los convertimos a números normales de Python con .numpy()
            data_row = [epoch + 1, avg_gen_loss.numpy(), avg_disc_loss.numpy(), avg_acc.numpy()]
            writer.writerow(data_row)
        
        # Al final de cada época, generamos y guardamos una muestra de imágenes.
        print(f'Generando imágenes de muestra para la época {epoch + 1}')
        generate_and_save_images(generator, epoch + 1, seed)

        print (f'Tiempo para la época {epoch + 1} es {time.time()-start} seg\n')

    # Generamos una última muestra al final del todo.
    generate_and_save_images(generator, epochs, seed)

# --- ¡EMPIEZA LA BATALLA! ---
print("--- Inciando entrenamiento ---")
train(train_dataset, EPOCHS)